In [141]:
import torch
import os
import json
import glob
from collections import defaultdict
from os import path
from transformers import BertTokenizer

In [142]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

I0211 13:18:27.436591 139886178457408 tokenization_utils.py:373] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/shtoshni/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


### HTML Conversion

In [143]:
HTML_START = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"></head>'

cluster_start_tag = '<div class="{}" style="border:2px; display : inline; border-style:solid; padding: {}px; border-color: {}; padding-right: 3px; padding-left: 3px">'
singleton_start_tag = '<div class="{}" style="border:2px; display : inline; border-style:dotted; padding:{}px; border-color: {}; padding-right: 3px; padding-left: 3px">'
end_tag = '</div>'

script_string =(
    """
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
<script>
$(document).ready(function(){
  $("div").mouseover(function(e){
      if( $(this).attr('class').match('cluster*') ) {
        $(document.getElementsByClassName($(e.target).attr('class'))).css("background-color", "#C0F3F9");
    }
  //}
  //}
    //  , function(e){
      //  $(document.getElementsByClassName($(e.target).attr('class'))).css("background-color", "#fff");
    });

  $("div").mouseout(function(e){
      if( $(this).attr('class').match('cluster*') ) {
        $(document.getElementsByClassName($(e.target).attr('class'))).css("background-color", "#fff");
    }
  //}
  //}
    //  , function(e){
      //  $(document.getElementsByClassName($(e.target).attr('class'))).css("background-color", "#fff");
    });

});
</script>
    """
    )



largest_padding = 13
padding_reduction = 3



In [144]:
def return_html(json_file):
    story_name = path.basename(story_file)
    print(story_name)
    
    with open(json_file) as f:
        doc = json.load(f)
    
    
    
    body_string ='<body><div class="wrapper" style="line-height: 3">'
    
    all_subtokens = []
    for sentence in doc["tokenized_doc"]["sentences"]:
        all_subtokens.extend(sentence)
    
    all_tokens = []
    cur_subtoken_list = []
    cur_idx = 0
    for idx, subtoken_idx in enumerate(doc["tokenized_doc"]["subtoken_map"]):
        if subtoken_idx == cur_idx:
            if all_subtokens[idx] not in ['[SEP]', '[CLS]']:
                cur_subtoken_list.append(all_subtokens[idx])
        else:
#             print(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
            all_tokens.append(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
            if all_subtokens[idx] not in ['[SEP]', '[CLS]']:
                cur_subtoken_list = [all_subtokens[idx]]
            else:
                cur_subtoken_list = []
            cur_idx = subtoken_idx
            
    
#     print(all_tokens[:30])
    if cur_subtoken_list != []:
        all_tokens.append(bert_tokenizer.convert_tokens_to_string(cur_subtoken_list))
    
#     print(len(doc["tokenized_doc"]["subtoken_map"]))
    print(len(all_tokens), doc["tokenized_doc"]["subtoken_map"][-1] + 1)
#     print(len(all_subtokens))
    
    ment_start_dict = defaultdict(list)
    ment_end_dict = defaultdict(list)
    cluster_idx_to_len = defaultdict(int)
    
    clusters = sorted(doc["clusters"], key=lambda x: x[0][0][0])
    
    for cluster_idx, ment_list in enumerate(clusters):
        ment_list  = sorted(ment_list, key=lambda x: x[0][0])
        for (ment_start, ment_end), string in ment_list:
            ment_start_dict[ment_start].append((ment_end, cluster_idx))
            ment_end_dict[ment_end].append((ment_start, cluster_idx))
            cluster_idx_to_len[cluster_idx] += 1
                        
    # Sort mentions with same mention start by later mention ends i.e. start with spans which are longer
    for ment_start in ment_start_dict:
        ment_start_dict[ment_start] = sorted(ment_start_dict[ment_start], key=lambda x: x[0], reverse=True)
        
    # Sort mentions with same mention end by later mention starts i.e. start with spans which are shorter
    for ment_end in ment_end_dict:
        ment_end_dict[ment_end] = sorted(ment_end_dict[ment_end], key=lambda x: x[0], reverse=True)
        
    active_clusters = 0
    cluster_seen = set()
    for token_idx, token in enumerate(all_tokens):
        token_added = False
        if token == "\n":
            body_string += "<br/>\n" 
            continue
        if token_idx in ment_start_dict:
            for (_, cluster_idx) in ment_start_dict[token_idx]:
                prefix = cluster_start_tag
                cluster_name = f"cluster_{str(cluster_idx).zfill(4)}"
                if cluster_idx_to_len[cluster_idx] == 1:
                    prefix = singleton_start_tag
                if cluster_idx in cluster_seen:
                    color = '#000000'
                else:
                    color = '#F64A8A'
                
                
                cluster_seen.add(cluster_idx)
                body_string += prefix.format(cluster_name, 
                    largest_padding - active_clusters * padding_reduction, color)
                active_clusters += 1
            
            body_string += token + " "
            token_added = True
        
        if not token_added:
            body_string += token + " "

        if token_idx in ment_end_dict:
            for (_, cluster_idx) in ment_end_dict[token_idx]:
                body_string += "<sub>" + str(cluster_idx).zfill(4) + "</sub>" + end_tag + " "
                active_clusters -= 1
                assert (active_clusters >= 0)
    
    html_string = HTML_START + script_string + body_string + "</div></body></html>"
    return html_string

### Process all files to get HTML version of the data

In [145]:
story_file = "/home/shtoshni/Research/long-doc-coref/notebooks/umem_ccarol_output.json"
book_html = return_html(story_file)

umem_ccarol_output.json
41698 41698


In [146]:
html_file = path.join("/home/shtoshni/Research/long-doc-coref/notebooks/", 
                      path.splitext(path.basename(story_file))[0] + ".html")

print(html_file)
with open(html_file, "w") as f:
    f.write(book_html)

/home/shtoshni/Research/long-doc-coref/notebooks/umem_ccarol_output.html
